In [1]:
import requests as r
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from time import sleep

In [2]:
def addFolder(path, folderName):
    if path[-1] != '/':
        path += '/'
    lsFolder = "ls {} | grep '{}'".format(path, folderName)
    lsFolder = !{lsFolder}
    if len(lsFolder) == 0:
        makeFolder = 'mkdir {}{}'.format(path, folderName)
        makeFolder = !{makeFolder}
    else:
        pathExists = lsFolder[0].find('ls: cannot') == -1
        if pathExists:
            raise IOError('Folder already exists.')
        else:
            raise Exception('Path does not exist.')

def ifNoLocalFileThenDownload(path, fileName, urlPrefix):
    if path[-1] != '/':
        path += '/'
        print('ifNoLocalFileThenDownload: correcting path to {}'.format(path))
    lsFile = "ls {} | grep '{}'".format(path, fileName)
    lsFile = !{lsFile}
    if len(lsFile) == 0:
        url = '/'.join((urlPrefix, fileName))
        print('getting file from url {} ... '.format(url), end='', flush=True)
        response = r.get(url)
        try:
            response.raise_for_status()
        except r.exceptions.HTTPError or \
               r.exceptions.ConnectionError or \
               r.exceptions.MaxRetryError or \
               NewConnectionError or \
               OSError:
            raise r.exceptions.HTTPError('Error getting file from url {}'.format(url))
        else:
            with open(path + fileName, 'wb') as f:
                f.write(response.content)
            print('done', flush=True)
        finally:
            sleep(1)
    else:
        pathExists = lsFile[0].find('ls: cannot access') == -1
        if not pathExists:
            raise Exception('Path does not exist.')
        print('already have file {}'.format(path + fileName))
    print('exiting successfully')

def parsePathSuffixsFromCompanyFile(companyFile, companyName, formIndex=2, formName='4'):
    pathSuffixs = []
    foundCompany = False
    for line in companyFile:
        ls = line.split()
        if len(ls) > 3:
            if ls[:formIndex] == companyName.split() and ls[formIndex] == formName:
                pathSuffixs.append(ls[-1])
                foundCompany = True
            elif foundCompany is True:
                break #sorted by company
    return pathSuffixs

In [3]:
#parameters
currYear = 2020
fileNameZip, fileName = 'company.zip', 'company.idx'
urlIndexPrefix = 'https://www.sec.gov/Archives/edgar/full-index'
urlFilePrefix = 'https://www.sec.gov/Archives'
companyName = 'BOEING CO'
formName = '4'
years = [i for i in range(1993, currYear+1)]
#years = [i for i in range(2018, 2019)] #test
quarters = ['QTR{}'.format(i+1) for i in range(4)]

In [ ]:
#grab quarterly company.idx files from SEC website;
#update local archive
try:
    addFolder('./', 'data')
except:
    pass
for year in years:
    #if necessary, make year folder
    try:
        addFolder('./data/', '{}'.format(year))
    except IOError as e:
        pass #folder already exists
    except Exception as e:
        print(e)
        raise #path dne
    for quarter in quarters:
        #if necessary, make quarter folder
        try:
            addFolder('./data/{}'.format(year), quarter)
        except IOError as e:
            pass #folder already exists
        except Exception as e:
            print(e)
            raise #path dne
            
        #if necessary, fetch company.zip from sec
        path = '/'.join(('./data', str(year), quarter, ''))
        url = '/'.join((urlIndexPrefix, str(year), quarter))
        try:
            ifNoLocalFileThenDownload(path, fileNameZip, url)
        except r.exceptions.HTTPError as e:
            print(e)
        except IOError as e:
            print(e)
        except Exception as e:
            print(e)

getting file from url https://www.sec.gov/Archives/edgar/full-index/1993/QTR1/company.zip ... done
exiting successfully
getting file from url https://www.sec.gov/Archives/edgar/full-index/1993/QTR2/company.zip ... done
exiting successfully
getting file from url https://www.sec.gov/Archives/edgar/full-index/1993/QTR3/company.zip ... done
exiting successfully
getting file from url https://www.sec.gov/Archives/edgar/full-index/1993/QTR4/company.zip ... done
exiting successfully
getting file from url https://www.sec.gov/Archives/edgar/full-index/1994/QTR1/company.zip ... done
exiting successfully
getting file from url https://www.sec.gov/Archives/edgar/full-index/1994/QTR2/company.zip ... done
exiting successfully
getting file from url https://www.sec.gov/Archives/edgar/full-index/1994/QTR3/company.zip ... done
exiting successfully
getting file from url https://www.sec.gov/Archives/edgar/full-index/1994/QTR4/company.zip ... done
exiting successfully
getting file from url https://www.sec.go

getting file from url https://www.sec.gov/Archives/edgar/full-index/2010/QTR1/company.zip ... done
exiting successfully
getting file from url https://www.sec.gov/Archives/edgar/full-index/2010/QTR2/company.zip ... done
exiting successfully
getting file from url https://www.sec.gov/Archives/edgar/full-index/2010/QTR3/company.zip ... done
exiting successfully
getting file from url https://www.sec.gov/Archives/edgar/full-index/2010/QTR4/company.zip ... done
exiting successfully
getting file from url https://www.sec.gov/Archives/edgar/full-index/2011/QTR1/company.zip ... done
exiting successfully
getting file from url https://www.sec.gov/Archives/edgar/full-index/2011/QTR2/company.zip ... done
exiting successfully
getting file from url https://www.sec.gov/Archives/edgar/full-index/2011/QTR3/company.zip ... done
exiting successfully
getting file from url https://www.sec.gov/Archives/edgar/full-index/2011/QTR4/company.zip ... done
exiting successfully
getting file from url https://www.sec.go

In [ ]:
#extract company.idx files
for year in years:
    for quarter in quarters:
        path = '/'.join(('./data', str(year), quarter, ''))
        unzipFile = 'unzip {} -d {}'.format(path + fileNameZip, path)
        unzipFile = !{unzipFile}
        if len(unzipFile) == 3:
            print('extracted file already exists')
        else:
            unzipSuccessful = unzipFile[0].find('unzip:') == -1
            if not unzipSuccessful:
                print('File {} unzip failed:\n\t{}'.format(path + fileNameZip, unzipFile))
            else:
                print('extracted file from {} to {}'.format(path + fileNameZip, 
                                                            unzipFile[1][unzipFile[1].find(':')+2:]))

In [ ]:
#grab form 4's for company companyName from SEC website
#as indexed by company.idx in year/quarter/ local files;
#update local archive
for year in years:
    for quarter in quarters:
        path = '/'.join(('./data', str(year), quarter, ''))
        try:
            with open(path + fileName, 'r') as f:
                pathSuffixs = parsePathSuffixsFromCompanyFile(f, companyName, formName=formName)
        except FileNotFoundError:
            print('no file {} found'.format(path + fileName))
        else:
            #if necessary ,make req'd directory
            pss = pathSuffixs[0].split(sep='/')
            try:
                addFolder('./data/', pss[2])
            except:
                pass

            #if necessary, fetch files from sec
            for pathSuffix in pathSuffixs:
                #pathSuffix form: 'edgar/data/.../....txt'
                pss = pathSuffix.split(sep='/')
                path = '/'.join(('.', *pss[1:-1], ''))
                textFileName = pss[-1]
                url = '/'.join((urlFilePrefix, *pss[:-1]))
                try:
                    ifNoLocalFileThenDownload(path, textFileName, url)
                except r.exceptions.HTTPError as e:
                    print(e)
                except IOError as e:
                    print(e)
                except Exception as e:
                    print(e)

In [ ]:
#clean up company.idx files
for year in years:
    for quarter in quarters:
        path = '/'.join(('./data', str(year), quarter, ''))
        cleanUpFile = 'rm {}'.format(path + fileName)
        cleanUpFile = !{cleanUpFile}
        cleanUpSuccessful = len(cleanUpFile) == 0
        if not cleanUpSuccessful:
            print('File {} does not exist'.format(path + fileName))
        else:
            print('cleaned file at {}'.format(path + fileName))